# Tensorboard

Tensorboard is a tool that we can use to visualise all sorts of things about our models. It was originally intended to be used with Google's Tensorflow framework but PyTorch now supports it since version 1.2.0.

## Running tensorboard in the notebook (not recommended)

If you want to visualise this stuff in a notebook, just run the "magic" commands shown below in a code cell in your notebook.
The % sign here is not valid Python syntax and as such will not work outside of a notebook. The availability of the commands depends on your IPython kernel.



In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

## Running tensorboard outside of a notebook

First you need to install it through pip by running `pip3 install tensorboard`

Next, navigate to the directory which your notebook file is in a terminal then run `tensorboard --logdir runs`. It should prompt you to open http://localhost:6006/ in a browser. When you do that, you'll be able to see tensorboard running there.

The ```--logdir runs``` tag tells tensorboard that you want the logs to be stored in a directory (hence logdir) called runs (you can store them anywhere else too).

Firstly, there will be no data here. During the training loop, we will populate the logging directory.

Tags visualise the data in different places on the GUI. For instance we might tag different curves for training and testing to visualise them in different graphs.

Runs group together data that was produced by the same writer. To separate out different runs, specify a new folder for the writer to store the data for that run in. We do this by specifying that filepath as an argument when we initilise the SummaryWriter.

In [12]:
import sys
sys.path.append('..')
from utils import get_dataloaders, NN
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import time

section = 'tensorboard_test'
graph_name = 'loss'

train_loader, _, _ = get_dataloaders()

# TRAINING LOOP
def train(model, epochs=1):

    optimiser = torch.optim.SGD(model.parameters(), lr=0.01)

    # CREATE AN OBJECT WHICH CAN WRITE TO TENSORBOARD
    run_id = time.time() # use the timestamp as a unique id for this run
    writer = SummaryWriter(log_dir=f'../../runs/{section}/{run_id}')
    batch_idx = 0

    for epoch in range(epochs):
        for batch in train_loader:

            # USUAL STUFF
            x, y = batch
            prediction = model(x)
            loss = F.cross_entropy(prediction, y)
            loss.backward()
            optimiser.step()
            optimiser.zero_grad()

            # WRITE THE LOSS TO TENSORBOARD
            writer.add_scalar(f'{section}/{graph_name}', loss.item(), batch_idx)
            batch_idx += 1

my_model = NN([784, 32, 10], distribution=True, flatten_input=True) # create a neural network. More on this later
train(my_model)

Once you run the above cell, you should be able to see the graph, in the specified section, in tensorboard.